In [1]:
pip install torchaudio

## Importing all neccessary libraries

In [2]:
import warnings
warnings. filterwarnings("ignore")
import os
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import torch.nn.functional as F
import torchaudio
from torchaudio import transforms as tfs
import numpy as np
import pandas as pd 
from torch.nn import Sequential as sq
from torch.nn import Linear as lnr
from torch import argmax as agm
from torch.nn import Module as ml


## Evaluation Metrics implementation (WER and CER)

In [3]:

def avg_wer(wer_scores, combined_ref_len):
    return float(sum(wer_scores)) / float(combined_ref_len)


def _levenshtein_distance(ref, hyp):

    m = len(ref)
    n = len(hyp)

    # special case
    if ref == hyp:
        return 0
    if m == 0:
        return n
    if n == 0:
        return m

    if m < n:
        ref, hyp = hyp, ref
        m, n = n, m

    # use O(min(m, n)) space
    distance = np.zeros((2, n + 1), dtype=np.int32)

    # initialize distance matrix
    for j in range(0,n + 1):
        distance[0][j] = j

    # calculate levenshtein distance
    for i in range(1, m + 1):
        prev_row_idx = (i - 1) % 2
        cur_row_idx = i % 2
        distance[cur_row_idx][0] = i
        for j in range(1, n + 1):
            if ref[i - 1] == hyp[j - 1]:
                distance[cur_row_idx][j] = distance[prev_row_idx][j - 1]
            else:
                s_num = distance[prev_row_idx][j - 1] + 1
                i_num = distance[cur_row_idx][j - 1] + 1
                d_num = distance[prev_row_idx][j] + 1
                distance[cur_row_idx][j] = min(s_num, i_num, d_num)

    return distance[m % 2][n]


def word_errors(reference, hypothesis, ignore_case=False, delimiter=' '):
    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    ref_words = reference.split(delimiter)
    hyp_words = hypothesis.split(delimiter)

    edit_distance = _levenshtein_distance(ref_words, hyp_words)
    return float(edit_distance), len(ref_words)


def char_errors(reference, hypothesis, ignore_case=False, remove_space=False):
    if ignore_case == True:
        reference = reference.lower()
        hypothesis = hypothesis.lower()

    join_char = ' '
    if remove_space == True:
        join_char = ''

    reference = join_char.join(filter(None, reference.split(' ')))
    hypothesis = join_char.join(filter(None, hypothesis.split(' ')))

    edit_distance = _levenshtein_distance(reference, hypothesis)
    return float(edit_distance), len(reference)


def wer(reference, hypothesis, ignore_case=False, delimiter=' '):
  
    edit_distance, ref_len = word_errors(reference, hypothesis, ignore_case,
                                         delimiter)

    if ref_len == 0:
        raise ValueError("Reference's word number should be greater than 0.")

    wer = float(edit_distance) / ref_len
    return wer


def cer(reference, hypothesis, ignore_case=False, remove_space=False):
 
    edit_distance, ref_len = char_errors(reference, hypothesis, ignore_case,
                                         remove_space)

    if ref_len == 0:
        raise ValueError("Length of reference should be greater than 0.")

    cer = float(edit_distance) / ref_len
    return cer

class TextTransform:
    def __init__(self):
        self.char_map={"'": 0, '': 1, 'a': 2, 'b': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'j': 11, 'k': 12, 'l': 13, 'm': 14, 'n': 15, 'o': 16, 'p': 17, 'q': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'v': 23, 'w': 24, 'x': 25, 'y': 26, 'z': 27}
        self.index_map={0: "'", 1: ' ', 2: 'a', 3: 'b', 4: 'c', 5: 'd', 6: 'e', 7: 'f', 8: 'g', 9: 'h', 10: 'i', 11: 'j', 12: 'k', 13: 'l', 14: 'm', 15: 'n', 16: 'o', 17: 'p', 18: 'q', 19: 'r', 20: 's', 21: 't', 22: 'u', 23: 'v', 24: 'w', 25: 'x', 26: 'y', 27: 'z'}
        
    def text_to_int(self, text):
        """ Use a character map and convert text to an integer sequence """
        int_sequence = []
        for c in text:
            if c == ' ':
                ch = self.char_map['']
            else:
                ch = self.char_map[c]
            int_sequence.append(ch)
        return int_sequence

    
    def int_to_text(self, labels):
        """ Use a character map and convert integer labels to an text sequence """
        string = []
        for i in labels:
            string.append(self.index_map[i])
        return ''.join(string).replace('', ' ')

train_audio_transforms = nn.Sequential(
    torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_mels=128),
    torchaudio.transforms.FrequencyMasking(freq_mask_param=30),
    torchaudio.transforms.TimeMasking(time_mask_param=100)
)

valid_audio_transforms = torchaudio.transforms.MelSpectrogram()

text_transform = TextTransform()

def data_processing(data, data_type="train"):
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    for (waveform, _, utterance, _, _, _) in data:
        if data_type == 'train':
            spec = train_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        elif data_type == 'valid':
            spec = valid_audio_transforms(waveform).squeeze(0).transpose(0, 1)
        else:
            raise Exception('data_type should be train or valid')
        spectrograms.append(spec)
        label = torch.Tensor(text_transform.text_to_int(utterance.lower()))
        labels.append(label)
        input_lengths.append(spec.shape[0]//2)
        label_lengths.append(len(label))

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)

    return spectrograms, labels, input_lengths, label_lengths


def GreedyDecoder(output, labels, label_lengths, blank_label=28, collapse_repeated=True):
	arg_maxes = torch.argmax(output, dim=2)
	decodes = []
	targets = []
	for i, args in enumerate(arg_maxes):
		decode = []
		targets.append(text_transform.int_to_text(labels[i][:label_lengths[i]].tolist()))
		for j, index in enumerate(args):
			if index != blank_label:
				if collapse_repeated and j != 0 and index == args[j -1]:
					continue
				decode.append(index.item())
		decodes.append(text_transform.int_to_text(decode))
	return decodes, targets

## The NVIDIA System Management Interface (management and monitoring of NVIDIA GPU devices)

In [4]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



# **CNN layer**

In [5]:
class CNNLayerNorm(nn.Module):
    def __init__(self, n_feats):
        super(CNNLayerNorm, self).__init__()
        self.layer_norm = nn.LayerNorm(n_feats)

    def forward(self, x):
        x = x.transpose(2, 3).contiguous() 
        x = self.layer_norm(x)
        return x.transpose(2, 3).contiguous() 


class ResidualCNN(nn.Module):

    def __init__(self, in_channels, out_channels, kernel, stride, dropout, n_feats):
        super(ResidualCNN, self).__init__()

        self.cnn1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding=kernel//2)
        self.cnn2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding=kernel//2)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.layer_norm1 = CNNLayerNorm(n_feats)
        self.layer_norm2 = CNNLayerNorm(n_feats)

    def forward(self, x):
        residual = x 
        x = self.layer_norm1(x)
        x = F.gelu(x)
        x = self.dropout1(x)
        x = self.cnn1(x)
        x = self.layer_norm2(x)
        x = F.gelu(x)
        x = self.dropout2(x)
        x = self.cnn2(x)
        x += residual
        return x 




# **Gated Recurrent Unit layer**

In [6]:
class BidirectionalGRU(nn.Module):

    def __init__(self, rnn_dim, hidden_size, dropout, batch_first):
        super(BidirectionalGRU, self).__init__()

        self.BiGRU = nn.GRU(
            input_size=rnn_dim, hidden_size=hidden_size,
            num_layers=1, batch_first=batch_first, bidirectional=True)
        self.layer_norm = nn.LayerNorm(rnn_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.layer_norm(x)
        x = F.gelu(x)
        x, _ = self.BiGRU(x)
        x = self.dropout(x)
        return x


# Final Automatic Speech Recognition Model architecture

In [7]:


class SpeechRecognitionModel(nn.Module):
    
    def __init__(self, n_cnn_layers, n_rnn_layers, rnn_dim, n_class, n_feats, stride=2, dropout=0.1):
        super(SpeechRecognitionModel, self).__init__()
        n_feats = n_feats//2
        self.cnn = nn.Conv2d(1, 32, 3, stride=stride, padding=3//2) 
        self.rescnn_layers = nn.Sequential(*[
            ResidualCNN(32, 32, kernel=3, stride=1, dropout=dropout, n_feats=n_feats) 
            for _ in range(n_cnn_layers)
        ])
        self.fully_connected = nn.Linear(n_feats*32, rnn_dim)
        self.birnn_layers = nn.Sequential(*[
            BidirectionalGRU(rnn_dim=rnn_dim if i==0 else rnn_dim*2,
                             hidden_size=rnn_dim, dropout=dropout, batch_first=i==0)
            for i in range(n_rnn_layers)
        ])
        self.classifier = nn.Sequential(
            nn.Linear(rnn_dim*2, rnn_dim),  
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(rnn_dim, n_class)
        )

    def forward(self, x):
        x = self.cnn(x)
        x = self.rescnn_layers(x)
        sizes = x.size()
        x = x.view(sizes[0], sizes[1] * sizes[2], sizes[3]) 
        x = x.transpose(1, 2) 
        x = self.fully_connected(x)
        x = self.birnn_layers(x)
        x = self.classifier(x)
        return x

# Training and Testing Code

In [8]:
class IterMeter(object):
    def __init__(self):
        self.val = 0

    def step(self):
        self.val += 1

    def get(self):
        return self.val


def train(model, device, train_loader, criterion, optimizer, scheduler, epoch, iter_meter):
    model.train()
    data_len = len(train_loader.dataset)

    for batch_idx, _data in enumerate(train_loader):
        spectrograms, labels, input_lengths, label_lengths = _data 
        spectrograms, labels = spectrograms.to(device), labels.to(device)

        optimizer.zero_grad()

        output = model(spectrograms) 
        output = F.log_softmax(output, dim=2)
        output = output.transpose(0, 1)

        loss = criterion(output, labels, input_lengths, label_lengths)
        loss.backward()

    
        optimizer.step()
        scheduler.step()
        iter_meter.step()
        if batch_idx % 100 == 0 or batch_idx == data_len:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(spectrograms), data_len,
                100. * batch_idx / len(train_loader), loss.item()))


def test(model, device, test_loader, criterion, epoch, iter_meter):
    print('\nevaluating...')
    model.eval()
    test_loss = 0
    test_cer, test_wer = [], []

    with torch.no_grad():
        for i, _data in enumerate(test_loader):
            spectrograms, labels, input_lengths, label_lengths = _data 
            spectrograms, labels = spectrograms.to(device), labels.to(device)

            output = model(spectrograms)  
            output = F.log_softmax(output, dim=2)
            output = output.transpose(0, 1)

            loss = criterion(output, labels, input_lengths, label_lengths)
            test_loss += loss.item() / len(test_loader)

            decoded_preds, decoded_targets = GreedyDecoder(output.transpose(0, 1), labels, label_lengths)
            for j in range(len(decoded_preds)):
                test_cer.append(cer(decoded_targets[j], decoded_preds[j]))
                test_wer.append(wer(decoded_targets[j], decoded_preds[j]))


    avg_cer = sum(test_cer)/len(test_cer)
    avg_wer = sum(test_wer)/len(test_wer)
  
    print('Test set: Average loss: {:.4f}, Average CER: {:4f} Average WER: {:.4f}\n'.format(test_loss, avg_cer, avg_wer))
    # filename='./best_fit_model'+str(epoch)+'.h5'
    # torch.save(model,filename)




# Hyperparameters of the model

In [9]:
learning_rate = 5e-4
batch_size = 5
epochs = 10
train_url = "train-clean-100"
test_url = "test-clean"

In [10]:
if not os.path.isdir("./data"):
    os.makedirs("./data")
train_dataset = torchaudio.datasets.LIBRISPEECH("./data", url=train_url, download=True)
test_dataset = torchaudio.datasets.LIBRISPEECH("./data", url=test_url, download=True)

In [11]:
hparams = {
    "n_cnn_layers": 3,
    "n_rnn_layers": 5,
    "rnn_dim": 512,
    "n_class": 29,
    "n_feats": 128,
    "stride":2,
    "dropout": 0.1,
    "learning_rate": learning_rate,
    "batch_size": batch_size,
    "epochs": epochs
}

use_cuda = torch.cuda.is_available()
torch.manual_seed(7)
device = torch.device("cuda" if use_cuda else "cpu")


kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = data.DataLoader(dataset=train_dataset,
                            batch_size=hparams['batch_size'],
                            shuffle=True,
                            collate_fn=lambda x: data_processing(x, 'train'),
                            **kwargs)
test_loader = data.DataLoader(dataset=test_dataset,
                            batch_size=hparams['batch_size'],
                            shuffle=False,
                            collate_fn=lambda x: data_processing(x, 'valid'),
                            **kwargs)

model = SpeechRecognitionModel(
    hparams['n_cnn_layers'], hparams['n_rnn_layers'], hparams['rnn_dim'],
    hparams['n_class'], hparams['n_feats'], hparams['stride'], hparams['dropout']
    ).to(device)

print(model)
print('Num Model Parameters', sum([param.nelement() for param in model.parameters()]))

optimizer = optim.AdamW(model.parameters(), hparams['learning_rate'])
criterion = nn.CTCLoss(blank=28).to(device)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=hparams['learning_rate'], steps_per_epoch=int(len(train_loader)),epochs=hparams['epochs'],anneal_strategy='linear')

iter_meter = IterMeter()


SpeechRecognitionModel(
  (cnn): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (rescnn_layers): Sequential(
    (0): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (layer_norm2): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
    )
    (1): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((64,),

# Code for model training and testing (Optional)


> This code snippet should only be used for base model training.

> Since we already have our based model trained , we will skip this part and load our trained model directly






In [12]:
# for epoch in range(1, epochs+1):
#     train(model, device, train_loader, criterion, optimizer, scheduler, epoch, iter_meter)
#     test(model, device, test_loader, criterion, epoch, iter_meter)

# Loading the best fit model


*   The base model for slurred speech Recognition has been trained on kaggle for 10 epoch.
*   It has been trained on Librispeech Corpus Speech Dataset



In [13]:
loaded_model = torch.load('/content/drive/MyDrive/BTP Datasets/Best_fit_model/best_fit_model10.h5',map_location=torch.device('cpu'))


In [14]:
print(loaded_model)

SpeechRecognitionModel(
  (cnn): Conv2d(1, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (rescnn_layers): Sequential(
    (0): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (layer_norm2): CNNLayerNorm(
        (layer_norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
    )
    (1): ResidualCNN(
      (cnn1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (cnn2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
      (layer_norm1): CNNLayerNorm(
        (layer_norm): LayerNorm((64,),

# Testing our loaded base model

In [15]:
test(loaded_model, device, test_loader, criterion, 1, iter_meter)


evaluating...
Test set: Average loss: 0.5171, Average CER: 0.119907 Average WER: 0.1413



In [17]:
import os
from typing import Tuple, Union
from pathlib import Path
import torchaudio
from torch import Tensor
from torch.utils.data import Dataset


def load_librispeech_item(fileid,path,ext_audio,ext_txt):
    speaker_id, chapter_id, utterance_id = fileid.split("-")

    file_text = speaker_id + "-" + chapter_id + ext_txt
    file_text = os.path.join(path, speaker_id, chapter_id, file_text)

    fileid_audio = speaker_id + "-" + chapter_id + "-" + utterance_id
    file_audio = fileid_audio + ext_audio
    file_audio = os.path.join(path, speaker_id, chapter_id, file_audio)

    waveform, sample_rate = torchaudio.load(file_audio)

    with open(file_text) as ft:
        for line in ft:
            fileid_text, utterance = line.strip().split(" ", 1)
            if fileid_audio == fileid_text:
                break
        else:
            raise FileNotFoundError("Translation not found for " + fileid_audio)

    return (waveform,sample_rate,utterance,int(speaker_id),int(chapter_id),int(utterance_id),)


## Custom class for loading Slurred Speech Test Dataset in Pytorch

In [18]:
class TestDataset(Dataset): 
    _ext_txt = ".trans.txt"
    _ext_audio = ".flac"
    def __init__(self):
        self._path="/content/drive/MyDrive/BTP Datasets/SlurredSpeech_Dataset_Test"
        self._walker=['1-2-0000', '1-2-0001', '1-2-0002', '1-2-0003', '1-2-0004', '1-2-0005', '1-2-0006', '1-2-0007', '1-2-0008', '1-2-0009', '1-2-0010', '1-2-0011', '1-2-0012', '1-2-0013', '1-2-0014', '1-2-0015', '1-2-0016', '1-2-0017', '1-2-0018', '1-2-0019', '1-2-0020', '1-2-0021', '1-2-0022', '1-2-0023', '1-2-0024', '1-2-0025', '1-2-0026', '1-2-0027', '1-2-0028', '1-2-0029', '1-2-0030', '1-2-0031', '1-2-0032', '1-2-0033', '1-2-0034', '1-2-0035', '1-2-0036', '1-2-0037', '1-2-0038', '1-2-0039', '1-2-0040', '1-2-0041', '1-2-0042', '1-2-0043', '1-2-0044', '1-2-0045', '1-2-0046', '1-2-0047', '1-2-0048', '1-2-0049', '1-2-0050', '1-2-0051', '1-2-0052', '1-2-0053', '1-2-0054', '1-2-0055', '1-2-0056', '1-2-0057', '1-2-0058', '1-2-0059', '1-2-0060', '1-2-0061', '1-2-0062', '1-2-0063', '1-2-0064', '1-2-0065', '1-2-0066', '1-2-0067', '1-2-0068', '1-2-0069', '1-2-0070', '1-2-0071', '1-2-0072', '1-2-0073', '1-2-0074', '1-2-0075', '1-2-0076', '1-2-0077', '1-2-0078', '1-2-0079', '1-2-0080', '1-2-0081', '1-2-0082', '1-2-0083', '1-2-0084', '1-2-0085', '1-2-0086', '1-2-0087', '1-2-0088', '1-2-0089', '1-2-0090', '1-2-0091', '1-2-0092', '1-2-0093', '1-2-0094', '1-2-0095', '1-2-0096', '1-2-0097', '1-2-0098', '1-2-0099']

    def __getitem__(self, n):
        fileid = self._walker[n]
        return load_librispeech_item(fileid, self._path, self._ext_audio, self._ext_txt)

    def __len__(self):
        return len(self._walker)   
    
    
testDataset_slurred_speech=TestDataset()

## Custom class for loading Slurred Speech Train Dataset in Pytorch

In [19]:
class TrainDataset(Dataset): 
    _ext_txt = ".trans.txt"
    _ext_audio = ".flac"
    def __init__(self):
        self._path="/content/drive/MyDrive/BTP Datasets/SlurredSpeech_Dataset_Train"
        self._walker=['1-2-0000', '1-2-0001', '1-2-0002', '1-2-0003', '1-2-0004', '1-2-0005', '1-2-0006', '1-2-0007', '1-2-0008', '1-2-0009', '1-2-0010', '1-2-0011', '1-2-0012', '1-2-0013', '1-2-0014', '1-2-0015', '1-2-0016', '1-2-0017', '1-2-0018', '1-2-0019', '1-2-0020', '1-2-0021', '1-2-0022', '1-2-0023', '1-2-0024', '1-2-0025', '1-2-0026', '1-2-0027', '1-2-0028', '1-2-0029', '1-2-0030', '1-2-0031', '1-2-0032', '1-2-0033', '1-2-0034', '1-2-0035', '1-2-0036', '1-2-0037', '1-2-0038', '1-2-0039', '1-2-0040', '1-2-0041', '1-2-0042', '1-2-0043', '1-2-0044', '1-2-0045', '1-2-0046', '1-2-0047', '1-2-0048', '1-2-0049', '1-2-0050', '1-2-0051', '1-2-0052', '1-2-0053', '1-2-0054', '1-2-0055', '1-2-0056', '1-2-0057', '1-2-0058', '1-2-0059', '1-2-0060', '1-2-0061', '1-2-0062', '1-2-0063', '1-2-0064', '1-2-0065', '1-2-0066', '1-2-0067', '1-2-0068', '1-2-0069', '1-2-0070', '1-2-0071', '1-2-0072', '1-2-0073', '1-2-0074', '1-2-0075', '1-2-0076', '1-2-0077', '1-2-0078', '1-2-0079', '1-2-0080', '1-2-0081', '1-2-0082', '1-2-0083', '1-2-0084', '1-2-0085', '1-2-0086', '1-2-0087', '1-2-0088', '1-2-0089', '1-2-0090', '1-2-0091', '1-2-0092', '1-2-0093', '1-2-0094', '1-2-0095', '1-2-0096', '1-2-0097', '1-2-0098', '1-2-0099']

    def __getitem__(self, n):
        fileid = self._walker[n]
        return load_librispeech_item(fileid, self._path, self._ext_audio, self._ext_txt)

    def __len__(self):
        return len(self._walker)   
    
    
trainDataset_slurred_speech=TrainDataset()

In [20]:

testDataLoaderSlurred = data.DataLoader(dataset=testDataset_slurred_speech,batch_size=hparams['batch_size'],shuffle=False,collate_fn=lambda x: data_processing(x, 'valid'),**kwargs)
trainDataLoderSlurred = data.DataLoader(dataset=trainDataset_slurred_speech,batch_size=hparams['batch_size'],shuffle=False,collate_fn=lambda x: data_processing(x, 'valid'),**kwargs)



# Testing our base model before fine tuning on slurred speech dataset

In [21]:
test(loaded_model, device, testDataLoaderSlurred, criterion, 1, iter_meter)


evaluating...
Test set: Average loss: 4.3672, Average CER: 0.638808 Average WER: 0.6565



# Applying transfer learning

In [ ]:
def transfer_learning_type1(pre_trained_model):
  hparams = {"learning_rate": 5e-4,"batch_size": 5,"epochs": 20}
  opti = optim.AdamW(pre_trained_model.parameters(), hparams['learning_rate'])
  crit = nn.CTCLoss(blank=28).to(device)
  sche = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=hparams['learning_rate'], steps_per_epoch=int(len(trainDataLoderSlurred)),epochs=hparams['epochs'],anneal_strategy='linear')
  iter = IterMeter()
  for epoch in range(1, epochs+1):
    train(pre_trained_model, device, trainDataLoderSlurred, crit, opti, sche, epoch, iter)
    test(pre_trained_model, device, testDataLoaderSlurred, crit, epoch, iter)

transfer_learning_type1(loaded_model)


In [22]:
params=loaded_model.state_dict()
print(params.keys())

odict_keys(['cnn.weight', 'cnn.bias', 'rescnn_layers.0.cnn1.weight', 'rescnn_layers.0.cnn1.bias', 'rescnn_layers.0.cnn2.weight', 'rescnn_layers.0.cnn2.bias', 'rescnn_layers.0.layer_norm1.layer_norm.weight', 'rescnn_layers.0.layer_norm1.layer_norm.bias', 'rescnn_layers.0.layer_norm2.layer_norm.weight', 'rescnn_layers.0.layer_norm2.layer_norm.bias', 'rescnn_layers.1.cnn1.weight', 'rescnn_layers.1.cnn1.bias', 'rescnn_layers.1.cnn2.weight', 'rescnn_layers.1.cnn2.bias', 'rescnn_layers.1.layer_norm1.layer_norm.weight', 'rescnn_layers.1.layer_norm1.layer_norm.bias', 'rescnn_layers.1.layer_norm2.layer_norm.weight', 'rescnn_layers.1.layer_norm2.layer_norm.bias', 'rescnn_layers.2.cnn1.weight', 'rescnn_layers.2.cnn1.bias', 'rescnn_layers.2.cnn2.weight', 'rescnn_layers.2.cnn2.bias', 'rescnn_layers.2.layer_norm1.layer_norm.weight', 'rescnn_layers.2.layer_norm1.layer_norm.bias', 'rescnn_layers.2.layer_norm2.layer_norm.weight', 'rescnn_layers.2.layer_norm2.layer_norm.bias', 'fully_connected.weight', 

In [24]:
def transfer_learning(pre_trained_model):
  learning_rate = 5e-4
  batch_size = 5
  epochs = 10
  params=loaded_model.state_dict()
  print(params.keys())
  loaded_model.cnn.weight.requires_grad=False
  loaded_model.rescnn_layers[0].cnn1.weight.requires_grad=False
  loaded_model.rescnn_layers[0].cnn2.weight.requires_grad=False
  loaded_model.rescnn_layers[0].layer_norm1.layer_norm.weight.requires_grad=False
  loaded_model.rescnn_layers[0].layer_norm2.layer_norm.weight.requires_grad=False
  loaded_model.rescnn_layers[1].cnn1.weight.requires_grad=False
  loaded_model.rescnn_layers[1].cnn2.weight.requires_grad=False
  hparams = {"n_cnn_layers": 3,"n_rnn_layers": 5,"rnn_dim": 512,"n_class": 29,"n_feats": 128,"stride":2,"dropout": 0.1,"learning_rate": learning_rate,"batch_size": batch_size,"epochs": epochs}
  opti = optim.AdamW(pre_trained_model.parameters(), hparams['learning_rate'])
  crit = nn.CTCLoss(blank=28).to(device)
  sche = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=hparams['learning_rate'], steps_per_epoch=int(len(trainDataLoderSlurred)),epochs=hparams['epochs'],anneal_strategy='linear')
  iter = IterMeter()
  for epoch in range(1, epochs+1):
    train(pre_trained_model, device, trainDataLoderSlurred, crit, opti, sche, epoch, iter)
    test(pre_trained_model, device, testDataLoaderSlurred, crit, epoch, iter)

transfer_learning(loaded_model)



odict_keys(['cnn.weight', 'cnn.bias', 'rescnn_layers.0.cnn1.weight', 'rescnn_layers.0.cnn1.bias', 'rescnn_layers.0.cnn2.weight', 'rescnn_layers.0.cnn2.bias', 'rescnn_layers.0.layer_norm1.layer_norm.weight', 'rescnn_layers.0.layer_norm1.layer_norm.bias', 'rescnn_layers.0.layer_norm2.layer_norm.weight', 'rescnn_layers.0.layer_norm2.layer_norm.bias', 'rescnn_layers.1.cnn1.weight', 'rescnn_layers.1.cnn1.bias', 'rescnn_layers.1.cnn2.weight', 'rescnn_layers.1.cnn2.bias', 'rescnn_layers.1.layer_norm1.layer_norm.weight', 'rescnn_layers.1.layer_norm1.layer_norm.bias', 'rescnn_layers.1.layer_norm2.layer_norm.weight', 'rescnn_layers.1.layer_norm2.layer_norm.bias', 'rescnn_layers.2.cnn1.weight', 'rescnn_layers.2.cnn1.bias', 'rescnn_layers.2.cnn2.weight', 'rescnn_layers.2.cnn2.bias', 'rescnn_layers.2.layer_norm1.layer_norm.weight', 'rescnn_layers.2.layer_norm1.layer_norm.bias', 'rescnn_layers.2.layer_norm2.layer_norm.weight', 'rescnn_layers.2.layer_norm2.layer_norm.bias', 'fully_connected.weight', 